# Example use of InterpletML

From the repo:

https://github.com/interpretml/interpret

namely:

https://nbviewer.jupyter.org/github/interpretml/interpret/blob/master/examples/python/notebooks/Explaining%20Blackbox%20Classifiers.ipynb

In [1]:
#!pip install interpret # will need to restart runtime

Some imports

In [2]:
# Needed to load and preprocess the data  
import pandas as pd
import numpy as np

# Some sklearn tools for preprocessing and building a pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# The algorithm used
from xgboost.sklearn import XGBClassifier


## Data

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("data/train.csv")
data.drop(["Id", "BUTTER"], axis=1, inplace=True)
features = data.columns
features = [feat.strip() for feat in features]
data.columns = features

sc = StandardScaler()


scaled = sc.fit_transform(data.iloc[:, :-1].values)

X = pd.DataFrame(data=scaled, columns=data.columns[:-1])
y = data.iloc[:, -1] 

X.head()

,B_OWNPV_CHI2,B_IPCHI2_OWNPV,B_FDCHI2_OWNPV,B_DIRA_OWNPV,B_PT,Kst_892_0_IP_OWNPV,Kst_892_0_cosThetaH,Kplus_IP_OWNPV,Kplus_P,piminus_IP_OWNPV,piminus_P,gamma_PT,piminus_ETA,Kplus_ETA
0,-0.132425,-0.076019,-0.020581,0.506625,2.457603,-0.010281,-1.635157,-0.000083,1.928913,0.092874,-0.567258,0.808782,-1.001080,-0.923161
1,0.215444,-1.235330,-0.192592,0.509224,-0.657723,-0.778374,-1.718573,-0.693486,0.495555,-0.968792,-0.734406,-0.702845,0.273690,-0.153645
2,0.337583,-0.175317,-0.199826,0.352163,-0.380149,-0.841548,0.066418,-0.833509,-0.103088,-0.749528,0.028360,-0.390404,0.544726,-0.085062
3,-1.080441,1.581900,-0.200571,-2.294296,-0.630449,-0.542046,0.772930,-0.627198,-0.995229,-0.421995,-0.747170,-0.925969,-1.647432,-1.832618
4,1.920171,2.166203,-0.208622,-0.839829,-0.926588,-0.884865,0.904880,-0.879123,-0.932438,-0.807563,0.141575,-0.657284,0.269023,0.218319


In [4]:
# stratify ensures test and train datasets have the same proportion of each class as y
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3, random_state=42)

In [5]:
features

['B_OWNPV_CHI2',
 'B_IPCHI2_OWNPV',
 'B_FDCHI2_OWNPV',
 'B_DIRA_OWNPV',
 'B_PT',
 'Kst_892_0_IP_OWNPV',
 'Kst_892_0_cosThetaH',
 'Kplus_IP_OWNPV',
 'Kplus_P',
 'piminus_IP_OWNPV',
 'piminus_P',
 'gamma_PT',
 'piminus_ETA',
 'Kplus_ETA',
 'signal']

## The model (XGBoost)

In [6]:
# XGBoost
xgb_model = Pipeline([#("preprocessor", preprocessor), 
                      # Add a scale_pos_weight to make it balanced
                      ("model", XGBClassifier(scale_pos_weight=(1 - y.mean()), n_jobs=-1))])

In [7]:
gs = GridSearchCV(xgb_model, {"model__max_depth": [5, 10],
                              "model__min_child_weight": [5, 10],
                              "model__n_estimators": [25]},
                  n_jobs=-1, cv=5, scoring="accuracy")

gs.fit(X_train, y_train)

/home/bambusito/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:20:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      min_child_weight=None,
                        

Best params and evaluation

In [8]:
print(gs.best_params_)
print(gs.best_score_)
xgb_model.set_params(**gs.best_params_)
xgb_model.fit(X_train, y_train)

{'model__max_depth': 10, 'model__min_child_weight': 10, 'model__n_estimators': 25}
0.7733822693374719
[21:20:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=10, min_child_weight=10, missing=nan,
                               monotone_constraints='()', n_estimators=25,
                               n_jobs=-1, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=0.665995805550592, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

## Using InterpretML

### Checking the data
The outcome is a drop-down menu for the “Summary” (the histogram of the target value) and each variable. When  choosing a variable, it shows the Pearson Correlation with the target value, followed by the histogram of the chosen variable in blue and the histogram of the target value in red.

In [9]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

<!-- http://127.0.0.1:7001/139760894492784/ -->

### Assessing performance

In [10]:
from interpret import show
from interpret.perf import ROC

blackbox_model = xgb_model
blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

<!-- http://127.0.0.1:7001/139759491344512/ -->

### Global interpretability


#### Global feature importance and Morris analysis of each feature

Useful for deciding which features to concentrate on.

In [11]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

<!-- http://127.0.0.1:7001/139759491895744/ -->

#### Partial dependence plots

Show the global relation of each feature to the target value (you have to select the feature on the widget). This is useful to decide how to transform important features: for instance, Kplus_P has a positive linear correlation with y from -1.3 to 0 (y ranging from 0.03 to 0.3), at the interval where it is more frequent, and starts saturating afterwards. This means that it mostly contributes to the noise (y=0) and that transforming it would be useless (since the relation is linear).

In [12]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

<!-- http://127.0.0.1:7001/139759491591616/ -->

### Local explanations

Via LIME and SHAP. Click on the predictions to see the corresponding explanation.

In [13]:
from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

<!-- http://127.0.0.1:7001/139759458996080/ -->

In [14]:
from interpret.blackbox import ShapKernel
import numpy as np

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=X_test.columns)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

<!-- http://127.0.0.1:7001/139757012057104/ -->

### Putting everything on a dahsboard

All the above information is available by navigating the dashboard.

In [15]:
show([marginal, blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])


<!-- http://127.0.0.1:7001/139757003113856/ -->
 Open in new window